**Фролов Владимир**

**Stepik ID: 541150734**

<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!unzip -q /content/gdrive/My\ Drive/simple_embeddings/SO_vectors_200 -d SO_vectors_200

In [ ]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("/content/SO_vectors_200/SO_vectors_200.bin_download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [ ]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [ ]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [ ]:
# method most_simmilar
def most_similar(words: list, embeddings, inclusion=False, print_negative=True):
    '''
    words: список слов, наличие которых проверяется в векторных представлениях;
    embeddings: массив векторных представлений;
    inclusion: проверяет входит ли inclusion в топ-5 близких слов к словам words,
    и если входит, то указывается место в рейтинге вхождений;
    print_negative: выводить или нет на печать отсутствующие слова.
    '''
    for word in words: 
        if word in embeddings:
            print(f'Слово {word} присутствует в векторных представлениях')
            similar = dict(embeddings.most_similar(word))
            if inclusion:
                try:
                    position = sorted(similar.values(), reverse=True).index(similar[inclusion]) + 1
                    if position <= 5: 
                        print(f'Слово {inclusion} входит в топ-5 близких слов к слову {word} под номером {position}')
                except KeyError:
                    print(f'Слово {inclusion} не входит в топ-5 близких слов к слову {word}')
        else:
            if print_negative:
                print(f'Слово {word} отсутствует в векторных представлениях')

In [ ]:
most_similar(['dog'], wv_embeddings, 'cat')

Слово dog присутствует в векторных представлениях
Слово cat не входит в топ-5 близких слов к слову dog


**Ответ: Получается, не входит.**

In [ ]:
most_similar(['cat'], wv_embeddings, 'dog')

Слово cat присутствует в векторных представлениях
Слово dog входит в топ-5 близких слов к слову cat под номером 2


А вот для `cat` слово `dog` является вторым по близости.

Проверим, приведены ли к нижнему регистру предобученные эмбеддинги, на словах, которые в английском языке обычно пишут с заглавной буквы:

In [ ]:
most_similar(['Apple', 'apple', 'Microsoft', 'microsoft'], wv_embeddings)

Слово Apple отсутствует в векторных представлениях
Слово apple присутствует в векторных представлениях
Слово Microsoft отсутствует в векторных представлениях
Слово microsoft присутствует в векторных представлениях


Перед обучением эмбеддингов слова были приведены к нижнему регистру. Получается, при поиске слов в векторных представлениях имеет смысл в дальнейшем приводить их к нижнему регистру. 

Проверим, были ли нормализованы слова перед обучением:

In [ ]:
most_similar(['abort', 'aborts', 'aborting', 'aborted'], wv_embeddings)

Слово abort присутствует в векторных представлениях
Слово aborts присутствует в векторных представлениях
Слово aborting присутствует в векторных представлениях
Слово aborted присутствует в векторных представлениях


Присутствуют различные формы одних и тех же слов, поэтому предварительная нормализация слов для создания токенов не обязательна. 

Присутствуют ли стоп-слова в векторных представлениях: 

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = list(stopwords.words('english'))
most_similar(stopwords, wv_embeddings, print_negative=False)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Слово ours присутствует в векторных представлениях
Слово himself присутствует в векторных представлениях
Слово will присутствует в векторных представлениях
Слово just присутствует в векторных представлениях
Слово now присутствует в векторных представлениях
Слово o присутствует в векторных представлениях
Слово y присутствует в векторных представлениях
Слово ain присутствует в векторных представлениях
Слово ma присутствует в векторных представлениях
Слово mightn присутствует в векторных представлениях
Слово needn присутствует в векторных представлениях


Только лишь малая часть стоп-слов присутствует в векторных представлениях, поэтому предварительная фильтрация по листу стоп-слов для создания токенов не обязательна.

Проверим, присутствуют ли знаки пунктуации в векторных представлениях:

In [ ]:
import string
most_similar(string.punctuation, wv_embeddings, print_negative=False)

Слово # присутствует в векторных представлениях
Слово + присутствует в векторных представлениях
Слово - присутствует в векторных представлениях
Слово / присутствует в векторных представлениях
Слово \ присутствует в векторных представлениях
Слово _ присутствует в векторных представлениях


В векторных представлениях присутствуют только несколько знаков пунктуации. 

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [ ]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)
tokenizer = MyTokenizer()

Попробуем также токенизаторы WP (помимо слов токенизирует знаки пунктуации без нормализации слов) и spacy (токенизирует в том числе знаки пунктуации и применяет нормализацию в виде лемматизации).

In [ ]:
from nltk.tokenize import WordPunctTokenizer
wp_tokenizer = WordPunctTokenizer()

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

class SpacyTokenizer():
    def __init__(self):
        pass
    def tokenize(self, text):
        spacy_results = nlp(text)
        return [token.lemma_ for token in spacy_results]

spacy_tokenizer = SpacyTokenizer()

In [ ]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """
    
    i = 0
    tokens = tokenizer.tokenize(question.lower()) # к нижнему регистру
    q_vec = np.zeros(dim, dtype='float32')
    for token in tokens:
        if token in embeddings:
            q_vec += embeddings[token]
            i += 1
    if i > 0:
        return q_vec / i # если есть хоть один эмбеддинг, то возвращаем среднее
    else:
        return q_vec # если нет, то возвращаем нулевой вектор

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [ ]:
tokenizers = {
    'default_tokenizer': tokenizer,
    'wp_tokenizer': wp_tokenizer,
    'spacy_tokenizer': spacy_tokenizer
}
string = 'I love neural networks'
for name, tokenizer in tokenizers.items():
    vec = question_to_vec(string, wv_embeddings, tokenizer)[2]
    print(f'{name}: {vec.round(2):.2f}.')

default_tokenizer: -1.29.
wp_tokenizer: -1.29.
spacy_tokenizer: -1.41.


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

Максимум разности этих двух метрик достигается при максимуме первого члена (уменьшаемое - `Hits@47`) и минимуме второго (вычитаемое - `DCG@1`). Максимум обеих метрик равен $1$ при $rank\_q_i^{'} = 1$ для каждого из $N$ вопросов вне зависимости от $K$ ($K$ не может быть меньше $1$). Однако для `Hits@K` для максимума достаточно, чтобы $rank\_q_i^{'}\le K$, т.к. в целом данная метрика не учитывает позицию дубликата в ранжированном списке, важно лишь выше объявленное условие. Минимум обеих метрик равен $0$ при $rank\_q_i^{'} > K$ для всех $N$. Если `Hits@47 - DCG@1` взаимосвязаны и относятся к одному примеру, то максимум их разности достигается при $2 \le rank\_q_{i}^{'} \le 47$ и равен $1$.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

[K = 10] $\text{DCG@10} = \frac{1}{\log_2(1+9)}\cdot[9 \le 10] = \frac{1}{\log_2{10}}$

In [ ]:
import math

K = 10
rank_dup = 9

print('\u2248', round((1 / math.log2(1 + rank_dup)) * (rank_dup <= K), 1))

≈ 0.3


### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [ ]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    hits = 0

    for rank in dup_ranks:
        hits += (rank <= k)
    
    hits_value = hits / len(dup_ranks)
    
    return hits_value    

In [ ]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    dcg = 0

    for rank in dup_ranks: 
        dcg += (1 / math.log2(1 + rank)) * (rank <= k)
    
    dcg_value = dcg / len(dup_ranks)
    
    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [ ]:
import pandas as pd

In [ ]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [candidates.index(question) + 1 for question, candidates in zip(copy_answers, candidates_ranking)]
print(f"{dup_ranks = }")

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

dup_ranks = [2]
Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [ ]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [ ]:
#import locale
#print(locale.getpreferredencoding())

ANSI_X3.4-1968


In [ ]:
#def getpreferredencoding(do_setlocale = True):
#    return "UTF-8"
#locale.getpreferredencoding = getpreferredencoding

In [ ]:
!unzip -q /content/gdrive/My\ Drive/simple_embeddings/stackoverflow_similar_questions -d stackoverflow_similar_questions

Считайте данные.

In [ ]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding='utf-8'):
        data.append(list(line.rstrip().split('\t')))
    return data

Нам понадобиться только файл validation.

In [ ]:
validation_data = read_corpus('/content/stackoverflow_similar_questions/data/validation.tsv')

Кол-во строк

In [ ]:
len(validation_data)

3760

Размер нескольких первых строк

In [ ]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [ ]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    q_vec = question_to_vec(question, embeddings, tokenizer, dim).reshape(1, -1)
    result = sorted(
        enumerate(candidates),
        key=lambda x: 1 - cosine_similarity(
            q_vec,
            question_to_vec(x[1], embeddings, tokenizer, dim).reshape(1, -1)
        )
    )
    
    zeros = 0
    if q_vec.all() == 0:
        zeros += 1
    
    return result, zeros

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [ ]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [ ]:
for name, tokenizer in tokenizers.items():
    for question, q_candidates, i in zip(questions, candidates, range(2)):
        ranks, zeros = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(f'Эксперимент №{i + 1} c {name}')
        print(f'Всего {zeros} нулевых векторов в вопросах')
        print(ranks)
        print()

Эксперимент №1 c default_tokenizer
Всего 0 нулевых векторов в вопросах
[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

Эксперимент №2 c default_tokenizer
Всего 0 нулевых векторов в вопросах
[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]

Эксперимент №1 c wp_tokenizer
Всего 0 нулевых векторов в вопросах
[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

Эксперимент №2 c wp_tokenizer
Всего 0 нулевых векторов в вопросах
[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]

Эксперимент №1 c spacy_tokenizer
Всего 0 нулевых

В экспериментах №1 и №2 все токенизаторы показали одинаковый результат. 

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

Последовательность начальных индексов вы должны получить `для эксперимента 2`  0, 2, 1.

Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
#results = [[(1, 'C# create cookie from string and send it'),
#            (0, 'Convert Google results object (pure js) to Python object'),
#            (2, 'How to use jQuery AJAX for an outside domain?')],
#           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
#            (*, 'select2 not displaying search results'), #скрыт
#            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

**Ответ: 021**

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
for name, tokenizer in tokenizers.items(): 
    sum_zero = 0
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in enumerate(tqdm(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks, zeros = rank_candidates(q, ex, wv_embeddings, tokenizer)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    
    print(f'C {name}:')
    print(f'Всего {zeros} нулевых векторов в вопросах')
    for k in tqdm([1, 5, 10, 100, 500, 1000]):
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))
    
    print()

  0%|          | 0/3760 [00:00<?, ?it/s]

C default_tokenizer:
Всего 0 нулевых векторов в вопросах


  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.415 | Hits@   1: 0.415
DCG@   5: 0.502 | Hits@   5: 0.582
DCG@  10: 0.525 | Hits@  10: 0.651
DCG@ 100: 0.570 | Hits@ 100: 0.874
DCG@ 500: 0.583 | Hits@ 500: 0.973
DCG@1000: 0.586 | Hits@1000: 1.000



  0%|          | 0/3760 [00:00<?, ?it/s]

...

DCG@   1: 0.410 | Hits@   1: 0.410
DCG@   5: 0.500 | Hits@   5: 0.580
DCG@  10: 0.521 | Hits@  10: 0.645
DCG@ 100: 0.568 | Hits@ 100: 0.875
DCG@ 500: 0.581 | Hits@ 500: 0.973
DCG@1000: 0.583 | Hits@1000: 1.000



  0%|          | 0/3760 [00:00<?, ?it/s]

C spacy_tokenizer:
Всего 0 нулевых векторов в вопросах


  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.391 | Hits@   1: 0.391
DCG@   5: 0.482 | Hits@   5: 0.562
DCG@  10: 0.501 | Hits@  10: 0.624
DCG@ 100: 0.549 | Hits@ 100: 0.853
DCG@ 500: 0.564 | Hits@ 500: 0.970
DCG@1000: 0.568 | Hits@1000: 1.000



На предобученных эмбеддингах лучшую точность показал дефолтный токенизатор на основе регулярных выражений. Чуть меньшую точность продемонстрировал `WordPunkt`-токенизатор, но в предобученных эмбеддингах мало знаков пунктуации, поэтому разница не слишком значительна. Минимальный результат показал токенизатор `Spacy` с лемматизацией. Видимо, нормализация (лемматизация) действительно не приносит пользы в данной задаче.  

### Эмбеддинги, обученные на корпусе похожих вопросов

In [ ]:
train_data = read_corpus('/content/stackoverflow_similar_questions/data/train.tsv')

In [ ]:
print(f'Количество строк:', len(train_data))
print()
for i in range(5):
    print(train_data[i])
    print()

Количество строк: 1000000

['converting string to list', 'Convert Google results object (pure js) to Python object']

['Which HTML 5 Canvas Javascript to use for making an interactive drawing tool?', 'Event handling for geometries in Three.js?']

['Sending array via Ajax fails', 'Getting all list items of an unordered list in PHP']

['How to insert CookieCollection to CookieContainer?', 'C# create cookie from string and send it']

['Updating one element of a bound Observable collection', 'WPF- How to update the changes in list item of a list']



Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [ ]:
import nltk
from nltk.corpus import stopwords
import string
from nltk.tokenize import word_tokenize
nltk.download('punkt')

stop_words = stopwords.words('english')
stopWords = set(stop_words)

def prepare_data(text):
    return ' '.join([word for word in word_tokenize(text.lower()) if word not in stopWords])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
words = []
for questions in tqdm(train_data):
    question = ' '.join(q for q in questions)
    words.append(prepare_data(question).split())

  0%|          | 0/1000000 [00:00<?, ?it/s]

Создадим токенизатор, в котором реализуется постпроцессинг: нормализация (стемминг), удаление стоп-слов, отсев знаков пунктуации, объединение некоторых последовательных токенов, подобных C и #. Последнее является, по сути, обратным преобразованием и неоднозначно, т.к. символ мог находится как в началле слова, так и в середине или конце.

In [ ]:
import re
from string import punctuation
from nltk.stem import PorterStemmer

ps = PorterStemmer()

soe_chars_within_tokens_without_forward_slash_and_hyphen = "_#+\\"
multi_char_punct_mark_chars = ['.'] 
word_token_chars = ''.join([re.escape(ch) for ch in soe_chars_within_tokens_without_forward_slash_and_hyphen]) + r"\w"
punct_multi_chars = ''.join([re.escape(ch) for ch in multi_char_punct_mark_chars])
punct_single_chars = ''.join([re.escape(ch) for ch in punctuation])
any_non_whitespace_chars = r"^\s"
token_pattern_v2 = re.compile(
    f"[{word_token_chars}]+|"
    f"[{punct_multi_chars}]+|"
    f"[{punct_single_chars}]|"
    f"[{any_non_whitespace_chars}]+"
    )

def filter_punctuation(tokens, punctuation_chars=punctuation):
    for token in tokens:
        if any(ch not in punctuation_chars for ch in token):
            yield token

def remove_stop_words(words, stop_words=stop_words):
    for word in words:
        if word not in stop_words:
            yield word

def stem(words, stemmer=ps.stem):
    for word in words:
        yield stemmer(word)

class MyTokenizer2:
    def __init__(self):
        pass
    def tokenize(self, text):
        t = token_pattern_v2.findall(text.lower())
        return list(stem(remove_stop_words(filter_punctuation(t))))

tokenizer_v2 = MyTokenizer2()

In [ ]:
text = "/sdlkgfjsdkl C# dir/ Windows\\System32 C++ c/c++ .cpp ... ?: e.g. --enable helloworld.py\tjohn\ndoe Руки-Ножницы needn/changes filter_pun"

In [ ]:
print(tokenizer_v2.tokenize(text))

['sdlkgfjsdkl', 'c#', 'dir', 'windows\\system32', 'c++', 'c', 'c++', 'cpp', 'e', 'g', 'enabl', 'helloworld', 'py', 'john', 'doe', 'руки', 'ножницы', 'chang', 'filter_pun']


Подберём оптимальные параметры `min_count` и `window` на примере второй версии токенизатора со `skip-gram`.

In [ ]:
from gensim.models import Word2Vec

In [ ]:
c_5 = []
for window in tqdm(range(3, 11)):
    embeddings_trained = Word2Vec(words,            # data for model to train on
                                  vector_size=200,  # embedding vector size
                                  min_count=5,      # consider words that occured at least 5 times
                                  window=window,    # windows size
                                  workers=8,        # num workers
                                  sg=1).wv          # algorithm
                                  
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in tqdm(enumerate(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks, _ = rank_candidates(q, ex, embeddings_trained, tokenizer_v2)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
        
    for k in [1, 5, 10, 100, 500, 1000]:
        c_5.append((5, window, k, dcg_score(wv_ranking, k), hits_count(wv_ranking, k)))

print(c_5)

  0%|          | 0/8 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

[(5, 3, 1, 0.384, 0.384), (5, 3, 5, 0.47480578899810877, 0.552), (5, 3, 10, 0.49407348543141616, 0.611), (5, 3, 100, 0.5348215181448404, 0.804), (5, 3, 500, 0.5523815547423199, 0.941), (5, 3, 1000, 0.5586060822760968, 1.0), (5, 4, 1, 0.396, 0.396), (5, 4, 5, 0.4860566562568039, 0.563), (5, 4, 10, 0.5036055598543355, 0.617), (5, 4, 100, 0.5437635573298318, 0.808), (5, 4, 500, 0.561174902813266, 0.943), (5, 4, 1000, 0.5671895410365224, 1.0), (5, 5, 1, 0.404, 0.404), (5, 5, 5, 0.4938337931694057, 0.571), (5, 5, 10, 0.5102513569314908, 0.622), (5, 5, 100, 0.5510379404349882, 0.816), (5, 5, 500, 0.5675923170479642, 0.944), (5, 5, 1000, 0.5734848579996307, 1.0), (5, 6, 1, 0.415, 0.415), (5, 6, 5, 0.4985571716312535, 0.57), (5, 6, 10, 0.51907392910595, 0.634), (5, 6, 100, 0.5576015516639514, 0.819), (5, 6, 500, 0.5743635994556027, 0.949), (5, 6, 1000, 0.5797149815877215, 1.0), (5, 7, 1, 0.415, 0.415), (5, 7, 5, 0.49966206019479215, 0.572), (5, 7, 10, 0.5189759270512654, 0.631), (5, 7, 100, 0.

In [ ]:
def get_best(result):
    count, best_window, best_score = 0, 0, 0
    
    for res in result:
        num_count, num_window, k, dcg, hits = res
        if k == 1 and hits > best_score:
            best_score = hits
            count = num_count
            best_window = num_window
    print(f"Для min_count = {count} лучший результат: Hits@1 = {best_score} при window = {best_window}") 

In [ ]:
get_best(c_5)

Для min_count = 5 лучший результат: Hits@1 = 0.418 при window = 10


In [ ]:
c_6 = []
for window in tqdm(range(3, 11)):
    embeddings_trained = Word2Vec(words,            # data for model to train on
                                  vector_size=200,  # embedding vector size
                                  min_count=6,      # consider words that occured at least 5 times
                                  window=window,    # windows size
                                  workers=8,        # num workers
                                  sg=1).wv          # algorithm
                                  
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in tqdm(enumerate(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks, _ = rank_candidates(q, ex, embeddings_trained, tokenizer_v2)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
        
    for k in [1, 5, 10, 100, 500, 1000]:
        c_6.append((6, window, k, dcg_score(wv_ranking, k), hits_count(wv_ranking, k)))

print(c_6)
get_best(c_6)

  0%|          | 0/8 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

[(6, 3, 1, 0.385, 0.385), (6, 3, 5, 0.47455704157578243, 0.55), (6, 3, 10, 0.49471496493676104, 0.612), (6, 3, 100, 0.5363608800280545, 0.81), (6, 3, 500, 0.553115920876861, 0.941), (6, 3, 1000, 0.5593356859842535, 1.0), (6, 4, 1, 0.392, 0.392), (6, 4, 5, 0.48650924435272774, 0.569), (6, 4, 10, 0.5034696114351448, 0.621), (6, 4, 100, 0.5445775987665914, 0.819), (6, 4, 500, 0.5610061524003058, 0.948), (6, 4, 1000, 0.5664695148761819, 1.0), (6, 5, 1, 0.397, 0.397), (6, 5, 5, 0.4885108688495631, 0.568), (6, 5, 10, 0.5050425644074399, 0.619), (6, 5, 100, 0.546985488033872, 0.819), (6, 5, 500, 0.5633706649985425, 0.946), (6, 5, 1000, 0.5690548757840379, 1.0), (6, 6, 1, 0.404, 0.404), (6, 6, 5, 0.49252632990976003, 0.566), (6, 6, 10, 0.5125543280422593, 0.627), (6, 6, 100, 0.5536574274991375, 0.822), (6, 6, 500, 0.570431831237731, 0.953), (6, 6, 1000, 0.5753355256835937, 1.0), (6, 7, 1, 0.402, 0.402), (6, 7, 5, 0.4988095594558083, 0.581), (6, 7, 10, 0.5157163857621491, 0.634), (6, 7, 100, 0.

In [ ]:
c_7 = []
for window in tqdm(range(3, 11)):
    embeddings_trained = Word2Vec(words,            # data for model to train on
                                  vector_size=200,  # embedding vector size
                                  min_count=7,      # consider words that occured at least 5 times
                                  window=window,    # windows size
                                  workers=8,        # num workers
                                  sg=1).wv          # algorithm
                                  
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in tqdm(enumerate(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks, _ = rank_candidates(q, ex, embeddings_trained, tokenizer_v2)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
        
    for k in [1, 5, 10, 100, 500, 1000]:
        c_7.append((7, window, k, dcg_score(wv_ranking, k), hits_count(wv_ranking, k)))

print(c_7)
print()
get_best(c_7)

  0%|          | 0/8 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

[(7, 3, 1, 0.383, 0.383), (7, 3, 5, 0.47916678359491066, 0.56), (7, 3, 10, 0.4969598376076185, 0.614), (7, 3, 100, 0.5372197837474438, 0.806), (7, 3, 500, 0.5547540397292452, 0.942), (7, 3, 1000, 0.560877951249964, 1.0), (7, 4, 1, 0.396, 0.396), (7, 4, 5, 0.4886786029026033, 0.57), (7, 4, 10, 0.5053383346772722, 0.622), (7, 4, 100, 0.5452832903230375, 0.813), (7, 4, 500, 0.5625605630006003, 0.947), (7, 4, 1000, 0.568133115089384, 1.0), (7, 5, 1, 0.404, 0.404), (7, 5, 5, 0.49250385007333114, 0.568), (7, 5, 10, 0.5114804346841613, 0.626), (7, 5, 100, 0.5509371225539655, 0.813), (7, 5, 500, 0.5686724824829519, 0.95), (7, 5, 1000, 0.5739227909422364, 1.0), (7, 6, 1, 0.416, 0.416), (7, 6, 5, 0.4991566652402572, 0.571), (7, 6, 10, 0.5196505502412879, 0.634), (7, 6, 100, 0.5590499298566773, 0.824), (7, 6, 500, 0.5753700800329612, 0.951), (7, 6, 1000, 0.5805127853796291, 1.0), (7, 7, 1, 0.416, 0.416), (7, 7, 5, 0.5012123350984223, 0.576), (7, 7, 10, 0.5206288423755902, 0.636), (7, 7, 100, 0.56

In [ ]:
c_8 = []
for window in tqdm(range(3, 11)):
    embeddings_trained = Word2Vec(words,            # data for model to train on
                                  vector_size=200,  # embedding vector size
                                  min_count=8,      # consider words that occured at least 5 times
                                  window=window,    # windows size
                                  workers=8,        # num workers
                                  sg=1).wv          # algorithm
                                  
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in tqdm(enumerate(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks, _ = rank_candidates(q, ex, embeddings_trained, tokenizer_v2)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
        
    for k in [1, 5, 10, 100, 500, 1000]:
        c_8.append((8, window, k, dcg_score(wv_ranking, k), hits_count(wv_ranking, k)))

print(c_8)
print()
get_best(c_8)

  0%|          | 0/8 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

[(8, 3, 1, 0.39, 0.39), (8, 3, 5, 0.47611395128694217, 0.548), (8, 3, 10, 0.4955148493093208, 0.607), (8, 3, 100, 0.5378145882142609, 0.808), (8, 3, 500, 0.5553838567444166, 0.945), (8, 3, 1000, 0.5611637267366988, 1.0), (8, 4, 1, 0.397, 0.397), (8, 4, 5, 0.4876162989120468, 0.566), (8, 4, 10, 0.5068404653637321, 0.626), (8, 4, 100, 0.5452528807242834, 0.81), (8, 4, 500, 0.5630572567237256, 0.947), (8, 4, 1000, 0.5686117676808462, 1.0), (8, 5, 1, 0.402, 0.402), (8, 5, 5, 0.4935724758363459, 0.571), (8, 5, 10, 0.5113309140144429, 0.626), (8, 5, 100, 0.5512023361929955, 0.814), (8, 5, 500, 0.5687449794641164, 0.949), (8, 5, 1000, 0.5740892600168447, 1.0), (8, 6, 1, 0.406, 0.406), (8, 6, 5, 0.4972680578807165, 0.575), (8, 6, 10, 0.5165911546939778, 0.635), (8, 6, 100, 0.5560877982638083, 0.826), (8, 6, 500, 0.5720441318297924, 0.951), (8, 6, 1000, 0.577183329721863, 1.0), (8, 7, 1, 0.412, 0.412), (8, 7, 5, 0.5015837804447268, 0.578), (8, 7, 10, 0.5202103109917519, 0.636), (8, 7, 100, 0.56

In [ ]:
c_9 = []
for window in tqdm(range(3, 11)):
    embeddings_trained = Word2Vec(words,            # data for model to train on
                                  vector_size=200,  # embedding vector size
                                  min_count=9,      # consider words that occured at least 5 times
                                  window=window,    # windows size
                                  workers=8,        # num workers
                                  sg=1).wv          # algorithm
                                  
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in tqdm(enumerate(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks, _ = rank_candidates(q, ex, embeddings_trained, tokenizer_v2)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
        
    for k in [1, 5, 10, 100, 500, 1000]:
        c_9.append((9, window, k, dcg_score(wv_ranking, k), hits_count(wv_ranking, k)))

print(c_9)
print()
get_best(c_9)

  0%|          | 0/8 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

[(9, 3, 1, 0.389, 0.389), (9, 3, 5, 0.47578457513917005, 0.549), (9, 3, 10, 0.49936815839524157, 0.622), (9, 3, 100, 0.5381509979517228, 0.811), (9, 3, 500, 0.5554473723691233, 0.946), (9, 3, 1000, 0.5611156788161412, 1.0), (9, 4, 1, 0.403, 0.403), (9, 4, 5, 0.4866798427555982, 0.558), (9, 4, 10, 0.5082266993569707, 0.624), (9, 4, 100, 0.5480216588350888, 0.813), (9, 4, 500, 0.5650577807195325, 0.945), (9, 4, 1000, 0.5708536300636442, 1.0), (9, 5, 1, 0.4, 0.4), (9, 5, 5, 0.49068508085502827, 0.569), (9, 5, 10, 0.5073891091644132, 0.62), (9, 5, 100, 0.5497880388652368, 0.82), (9, 5, 500, 0.5657906459927223, 0.944), (9, 5, 1000, 0.5716964140016774, 1.0), (9, 6, 1, 0.413, 0.413), (9, 6, 5, 0.4994794327049955, 0.575), (9, 6, 10, 0.5177711507181534, 0.632), (9, 6, 100, 0.5581517758796418, 0.824), (9, 6, 500, 0.5748480853027462, 0.954), (9, 6, 1000, 0.5796585573624398, 1.0), (9, 7, 1, 0.414, 0.414), (9, 7, 5, 0.50189483185761, 0.578), (9, 7, 10, 0.5219780717751201, 0.64), (9, 7, 100, 0.56094

In [ ]:
c_10 = []
for window in tqdm(range(3, 11)):
    embeddings_trained = Word2Vec(words,            # data for model to train on
                                  vector_size=200,  # embedding vector size
                                  min_count=10,     # consider words that occured at least 5 times
                                  window=window,    # windows size
                                  workers=8,        # num workers
                                  sg=1).wv          # algorithm
                                  
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in tqdm(enumerate(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks, _ = rank_candidates(q, ex, embeddings_trained, tokenizer_v2)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
        
    for k in [1, 5, 10, 100, 500, 1000]:
        c_10.append((10, window, k, dcg_score(wv_ranking, k), hits_count(wv_ranking, k)))

print(c_10)
print()
get_best(c_10)

  0%|          | 0/8 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

[(10, 3, 1, 0.39, 0.39), (10, 3, 5, 0.48156421720711906, 0.559), (10, 3, 10, 0.49870828776717335, 0.611), (10, 3, 100, 0.5406828547556026, 0.81), (10, 3, 500, 0.5580861133101863, 0.946), (10, 3, 1000, 0.5637525231427211, 1.0), (10, 4, 1, 0.395, 0.395), (10, 4, 5, 0.4833386205004998, 0.559), (10, 4, 10, 0.504664611121262, 0.624), (10, 4, 100, 0.5450430268888609, 0.818), (10, 4, 500, 0.561586512983238, 0.947), (10, 4, 1000, 0.5671475051790492, 1.0), (10, 5, 1, 0.405, 0.405), (10, 5, 5, 0.49344281206026464, 0.566), (10, 5, 10, 0.5136862480605308, 0.628), (10, 5, 100, 0.5545424076880506, 0.822), (10, 5, 500, 0.570998453120923, 0.95), (10, 5, 1000, 0.5762445964449118, 1.0), (10, 6, 1, 0.41, 0.41), (10, 6, 5, 0.5000334787414631, 0.578), (10, 6, 10, 0.5181341643999043, 0.634), (10, 6, 100, 0.5573616901787075, 0.822), (10, 6, 500, 0.5744412583748741, 0.954), (10, 6, 1000, 0.579254818225236, 1.0), (10, 7, 1, 0.416, 0.416), (10, 7, 5, 0.5014101099382068, 0.574), (10, 7, 10, 0.5222661966547056, 0

Подберём оптимальные параметры `min_count` и `window` на примере второй версии токенизатора с `CBOW` (Continuous Bag of Words).

In [ ]:
mc_5 = []
for window in tqdm(range(3, 11)):
    embeddings_trained = Word2Vec(words,            # data for model to train on
                                  vector_size=200,  # embedding vector size
                                  min_count=5,      # consider words that occured at least 5 times
                                  window=window,    # windows size
                                  workers=8,        # num workers
                                  sg=0).wv          # algorithm
                                  
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in tqdm(enumerate(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks, _ = rank_candidates(q, ex, embeddings_trained, tokenizer_v2)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
        
    for k in [1, 5, 10, 100, 500, 1000]:
        mc_5.append((5, window, k, dcg_score(wv_ranking, k), hits_count(wv_ranking, k)))

print(mc_5)
print()
get_best(mc_5)

  0%|          | 0/8 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

[(5, 3, 1, 0.306, 0.306), (5, 3, 5, 0.39530106492285183, 0.476), (5, 3, 10, 0.416938596775573, 0.543), (5, 3, 100, 0.4691680319289343, 0.797), (5, 3, 500, 0.48705489184697565, 0.935), (5, 3, 1000, 0.4939597645937497, 1.0), (5, 4, 1, 0.303, 0.303), (5, 4, 5, 0.3933585422574522, 0.474), (5, 4, 10, 0.41774608776158045, 0.549), (5, 4, 100, 0.4686094429021752, 0.795), (5, 4, 500, 0.48669442214433023, 0.934), (5, 4, 1000, 0.49371762518164086, 1.0), (5, 5, 1, 0.306, 0.306), (5, 5, 5, 0.3952484685186126, 0.471), (5, 5, 10, 0.4213648578154963, 0.551), (5, 5, 100, 0.4718584142786174, 0.796), (5, 5, 500, 0.4899059774312769, 0.935), (5, 5, 1000, 0.49681110878543355, 1.0), (5, 6, 1, 0.304, 0.304), (5, 6, 5, 0.3988023576999757, 0.48), (5, 6, 10, 0.4252505071282825, 0.562), (5, 6, 100, 0.4728594992334651, 0.793), (5, 6, 500, 0.49143030787408004, 0.936), (5, 6, 1000, 0.4982382688212546, 1.0), (5, 7, 1, 0.314, 0.314), (5, 7, 5, 0.40787219584121387, 0.491), (5, 7, 10, 0.43072430699781156, 0.562), (5, 7,

In [ ]:
mc_6 = []
for window in tqdm(range(3, 11)):
    embeddings_trained = Word2Vec(words,            # data for model to train on
                                  vector_size=200,  # embedding vector size
                                  min_count=6,      # consider words that occured at least 5 times
                                  window=window,    # windows size
                                  workers=8,        # num workers
                                  sg=0).wv          # algorithm
                                  
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in tqdm(enumerate(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks, _ = rank_candidates(q, ex, embeddings_trained, tokenizer_v2)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
        
    for k in [1, 5, 10, 100, 500, 1000]:
        mc_6.append((6, window, k, dcg_score(wv_ranking, k), hits_count(wv_ranking, k)))

print(mc_6)
print()
get_best(mc_6)

  0%|          | 0/8 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

[(6, 3, 1, 0.298, 0.298), (6, 3, 5, 0.3916654914929243, 0.473), (6, 3, 10, 0.4152669432055438, 0.546), (6, 3, 100, 0.4672601035939868, 0.799), (6, 3, 500, 0.4847113301132956, 0.934), (6, 3, 1000, 0.49173034053380293, 1.0), (6, 4, 1, 0.309, 0.309), (6, 4, 5, 0.3951298995716789, 0.472), (6, 4, 10, 0.41929860028020616, 0.546), (6, 4, 100, 0.4704883514473978, 0.794), (6, 4, 500, 0.4889335467544437, 0.936), (6, 4, 1000, 0.49575815633303505, 1.0), (6, 5, 1, 0.306, 0.306), (6, 5, 5, 0.4010245224333242, 0.483), (6, 5, 10, 0.4235495616658396, 0.552), (6, 5, 100, 0.4740241172466064, 0.797), (6, 5, 500, 0.49176770947382736, 0.933), (6, 5, 1000, 0.49891664675920017, 1.0), (6, 6, 1, 0.309, 0.309), (6, 6, 5, 0.4030043908971377, 0.486), (6, 6, 10, 0.42740441552738684, 0.562), (6, 6, 100, 0.4750246744372965, 0.794), (6, 6, 500, 0.493577960692867, 0.937), (6, 6, 1000, 0.5002743394442609, 1.0), (6, 7, 1, 0.312, 0.312), (6, 7, 5, 0.40656904386307474, 0.489), (6, 7, 10, 0.4297636279050771, 0.561), (6, 7, 

NameError: ignored

In [ ]:
get_best(mc_6)

Для min_count = 6 лучший результат: Hits@1 = 0.319 при window = 10


In [ ]:
mc_7 = []
for window in tqdm(range(3, 11)):
    embeddings_trained = Word2Vec(words,            # data for model to train on
                                  vector_size=200,  # embedding vector size
                                  min_count=7,      # consider words that occured at least 5 times
                                  window=window,    # windows size
                                  workers=8,        # num workers
                                  sg=0).wv          # algorithm
                                  
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in tqdm(enumerate(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks, _ = rank_candidates(q, ex, embeddings_trained, tokenizer_v2)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
        
    for k in [1, 5, 10, 100, 500, 1000]:
        mc_7.append((7, window, k, dcg_score(wv_ranking, k), hits_count(wv_ranking, k)))

print(mc_7)
print()
get_best(mc_7)

  0%|          | 0/8 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

[(7, 3, 1, 0.301, 0.301), (7, 3, 5, 0.3935144034035329, 0.479), (7, 3, 10, 0.4156690762818737, 0.549), (7, 3, 100, 0.46766594735465467, 0.804), (7, 3, 500, 0.48480604975422287, 0.938), (7, 3, 1000, 0.49139074199117644, 1.0), (7, 4, 1, 0.305, 0.305), (7, 4, 5, 0.39568451323158715, 0.475), (7, 4, 10, 0.41845450737834833, 0.545), (7, 4, 100, 0.4712496740659221, 0.801), (7, 4, 500, 0.4891073149269435, 0.939), (7, 4, 1000, 0.4955941929337452, 1.0), (7, 5, 1, 0.313, 0.313), (7, 5, 5, 0.4002159225765213, 0.478), (7, 5, 10, 0.4240451354483033, 0.551), (7, 5, 100, 0.47490068884595116, 0.797), (7, 5, 500, 0.4930561643573819, 0.937), (7, 5, 1000, 0.49977642084246904, 1.0), (7, 6, 1, 0.312, 0.312), (7, 6, 5, 0.40375672647270133, 0.484), (7, 6, 10, 0.42843991264206693, 0.561), (7, 6, 100, 0.47663029211057895, 0.796), (7, 6, 500, 0.494926072199328, 0.937), (7, 6, 1000, 0.501628942890552, 1.0), (7, 7, 1, 0.311, 0.311), (7, 7, 5, 0.40399613294304887, 0.484), (7, 7, 10, 0.4295006086689845, 0.563), (7, 

In [ ]:
mc_8 = []
for window in tqdm(range(3, 11)):
    embeddings_trained = Word2Vec(words,            # data for model to train on
                                  vector_size=200,  # embedding vector size
                                  min_count=8,      # consider words that occured at least 5 times
                                  window=window,    # windows size
                                  workers=8,        # num workers
                                  sg=0).wv          # algorithm
                                  
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in tqdm(enumerate(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks, _ = rank_candidates(q, ex, embeddings_trained, tokenizer_v2)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
        
    for k in [1, 5, 10, 100, 500, 1000]:
        mc_8.append((8, window, k, dcg_score(wv_ranking, k), hits_count(wv_ranking, k)))

print(mc_8)
print()
get_best(mc_8)

  0%|          | 0/8 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

[(8, 3, 1, 0.299, 0.299), (8, 3, 5, 0.39111573128860555, 0.472), (8, 3, 10, 0.4147391589650252, 0.545), (8, 3, 100, 0.46764267801517856, 0.801), (8, 3, 500, 0.48555554801001094, 0.941), (8, 3, 1000, 0.49181919212718234, 1.0), (8, 4, 1, 0.309, 0.309), (8, 4, 5, 0.39698121405693493, 0.473), (8, 4, 10, 0.42025548693963305, 0.545), (8, 4, 100, 0.4727636122031043, 0.8), (8, 4, 500, 0.4905417113210102, 0.938), (8, 4, 1000, 0.49712958330715024, 1.0), (8, 5, 1, 0.307, 0.307), (8, 5, 5, 0.3992644436029832, 0.48), (8, 5, 10, 0.42394893824311436, 0.556), (8, 5, 100, 0.4725959441591719, 0.793), (8, 5, 500, 0.49126495355027155, 0.937), (8, 5, 1000, 0.49796129473779727, 1.0), (8, 6, 1, 0.313, 0.313), (8, 6, 5, 0.4019108076171184, 0.479), (8, 6, 10, 0.4272144116932895, 0.557), (8, 6, 100, 0.4755358650537138, 0.793), (8, 6, 500, 0.49420290100336417, 0.937), (8, 6, 1000, 0.5009017328089597, 1.0), (8, 7, 1, 0.315, 0.315), (8, 7, 5, 0.4071536179108268, 0.491), (8, 7, 10, 0.4305867525275615, 0.563), (8, 7

In [ ]:
mc_9 = []
for window in tqdm(range(3, 11)):
    embeddings_trained = Word2Vec(words,            # data for model to train on
                                  vector_size=200,  # embedding vector size
                                  min_count=9,      # consider words that occured at least 5 times
                                  window=window,    # windows size
                                  workers=8,        # num workers
                                  sg=0).wv          # algorithm
                                  
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in tqdm(enumerate(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks, _ = rank_candidates(q, ex, embeddings_trained, tokenizer_v2)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
        
    for k in [1, 5, 10, 100, 500, 1000]:
        mc_9.append((9, window, k, dcg_score(wv_ranking, k), hits_count(wv_ranking, k)))

print(mc_9)
print()
get_best(mc_9)

  0%|          | 0/8 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

[(9, 3, 1, 0.305, 0.305), (9, 3, 5, 0.39585511095931963, 0.477), (9, 3, 10, 0.41712527633476504, 0.543), (9, 3, 100, 0.46890678845925493, 0.794), (9, 3, 500, 0.48719268049316283, 0.935), (9, 3, 1000, 0.4941025965130071, 1.0), (9, 4, 1, 0.31, 0.31), (9, 4, 5, 0.3956660061922361, 0.472), (9, 4, 10, 0.4199272307730789, 0.547), (9, 4, 100, 0.47101968497500984, 0.794), (9, 4, 500, 0.48975774233686964, 0.938), (9, 4, 1000, 0.49634128616609297, 1.0), (9, 5, 1, 0.306, 0.306), (9, 5, 5, 0.39793973860633863, 0.478), (9, 5, 10, 0.422753590679324, 0.555), (9, 5, 100, 0.4720597745109107, 0.795), (9, 5, 500, 0.4902519931131069, 0.935), (9, 5, 1000, 0.49716557248368803, 1.0), (9, 6, 1, 0.309, 0.309), (9, 6, 5, 0.3996767961013055, 0.479), (9, 6, 10, 0.4253507121513572, 0.558), (9, 6, 100, 0.47451806504110533, 0.797), (9, 6, 500, 0.4928286204027865, 0.939), (9, 6, 1000, 0.49929485625231895, 1.0), (9, 7, 1, 0.311, 0.311), (9, 7, 5, 0.4046152165892941, 0.484), (9, 7, 10, 0.42949421648565184, 0.561), (9, 

In [ ]:
mc_10 = []
for window in tqdm(range(3, 11)):
    embeddings_trained = Word2Vec(words,            # data for model to train on
                                  vector_size=200,  # embedding vector size
                                  min_count=10,     # consider words that occured at least 5 times
                                  window=window,    # windows size
                                  workers=8,        # num workers
                                  sg=0).wv          # algorithm
                                  
    wv_ranking = []
    max_validation_examples = 1000
    for i, line in tqdm(enumerate(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks, _ = rank_candidates(q, ex, embeddings_trained, tokenizer_v2)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
        
    for k in [1, 5, 10, 100, 500, 1000]:
        mc_10.append((10, window, k, dcg_score(wv_ranking, k), hits_count(wv_ranking, k)))

print(mc_10)
print()
get_best(mc_10)

  0%|          | 0/8 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

[(10, 3, 1, 0.305, 0.305), (10, 3, 5, 0.3950948759490119, 0.476), (10, 3, 10, 0.4192906478796158, 0.552), (10, 3, 100, 0.468903619680084, 0.795), (10, 3, 500, 0.4872926447178232, 0.937), (10, 3, 1000, 0.4939861437275098, 1.0), (10, 4, 1, 0.306, 0.306), (10, 4, 5, 0.3955618421072426, 0.474), (10, 4, 10, 0.4204765388209736, 0.552), (10, 4, 100, 0.47071162221009766, 0.798), (10, 4, 500, 0.488892786341934, 0.939), (10, 4, 1000, 0.4953697735552361, 1.0), (10, 5, 1, 0.309, 0.309), (10, 5, 5, 0.39679280696876296, 0.474), (10, 5, 10, 0.42259635969557974, 0.553), (10, 5, 100, 0.47196897603585775, 0.791), (10, 5, 500, 0.49144165240572907, 0.941), (10, 5, 1000, 0.49770074531003905, 1.0), (10, 6, 1, 0.31, 0.31), (10, 6, 5, 0.40252754228798293, 0.484), (10, 6, 10, 0.42766285514622276, 0.562), (10, 6, 100, 0.47569585110591156, 0.798), (10, 6, 500, 0.49410996080379255, 0.941), (10, 6, 1000, 0.5003748369665574, 1.0), (10, 7, 1, 0.309, 0.309), (10, 7, 5, 0.40379533757346775, 0.488), (10, 7, 10, 0.42685

Лучший результат достигнут при `min_count = 8`, `window = 10` и алгоритме `skip-gram`. Однако если `window = 10` - это ещё приемлемо, то `min_count = 8`, возможно, не совсем хороший выбор, т.к. в векторных представлениях будет мало слов. 

Но всё же попробуем использовать полученные оптимальные параметры и проверим качество ранжирования для каждого из четырёх токенизаторов: 

In [ ]:
embeddings_trained = Word2Vec(words,            # data for model to train on
                              vector_size=200,  # embedding vector size
                              min_count=8,      # consider words that occured at least 5 times
                              window=10,        # windows size
                              workers=8,        # num workers
                              sg=1).wv          # algorithm

In [ ]:
tokenizers = {
    'default_tokenizer': tokenizer,
    'tokenizer v.2': tokenizer_v2,
    'wp_tokenizer': wp_tokenizer,
    'spacy_tokenizer': spacy_tokenizer
}

In [ ]:
sum_zero = 0
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks, zeros = rank_candidates(q, ex, embeddings_trained, tokenizer)
    sum_zero += zeros
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)
print(f'C default tokenizer:')
print(f'Всего {sum_zero} нулевых векторов в вопросах')
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

print()

  0%|          | 0/3760 [00:00<?, ?it/s]

C default tokenizer:
Всего 1 нулевых векторов в вопросах
DCG@   1: 0.513 | Hits@   1: 0.513
DCG@   5: 0.597 | Hits@   5: 0.673
DCG@  10: 0.618 | Hits@  10: 0.737
DCG@ 100: 0.651 | Hits@ 100: 0.893
DCG@ 500: 0.661 | Hits@ 500: 0.976
DCG@1000: 0.664 | Hits@1000: 1.000



In [ ]:
sum_zero = 0
wv2_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks, zeros = rank_candidates(q, ex, embeddings_trained, tokenizer_v2)
    sum_zero += zeros
    wv2_ranking.append([r[0] for r in ranks].index(0) + 1)
print(f'C tokenizer v.2:')
print(f'Всего {sum_zero} нулевых векторов в вопросах')
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv2_ranking, k), k, hits_count(wv2_ranking, k)))

print()

  0%|          | 0/3760 [00:00<?, ?it/s]

C tokenizer v.2:
Всего 5 нулевых векторов в вопросах
DCG@   1: 0.414 | Hits@   1: 0.414
DCG@   5: 0.507 | Hits@   5: 0.587
DCG@  10: 0.526 | Hits@  10: 0.645
DCG@ 100: 0.565 | Hits@ 100: 0.831
DCG@ 500: 0.580 | Hits@ 500: 0.950
DCG@1000: 0.585 | Hits@1000: 1.000



In [ ]:
sum_zero = 0
wp_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks, zeros = rank_candidates(q, ex, embeddings_trained, wp_tokenizer)
    sum_zero += zeros
    wp_ranking.append([r[0] for r in ranks].index(0) + 1)
print(f'C wp tokenizer:')
print(f'Всего {sum_zero} нулевых векторов в вопросах')
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wp_ranking, k), k, hits_count(wp_ranking, k)))

print()

  0%|          | 0/3760 [00:00<?, ?it/s]

C wp tokenizer:
Всего 0 нулевых векторов в вопросах
DCG@   1: 0.517 | Hits@   1: 0.517
DCG@   5: 0.603 | Hits@   5: 0.677
DCG@  10: 0.623 | Hits@  10: 0.741
DCG@ 100: 0.657 | Hits@ 100: 0.902
DCG@ 500: 0.667 | Hits@ 500: 0.980
DCG@1000: 0.669 | Hits@1000: 1.000



In [ ]:
sum_zero = 0
sp_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks, zeros = rank_candidates(q, ex, embeddings_trained, spacy_tokenizer)
    sum_zero += zeros
    sp_ranking.append([r[0] for r in ranks].index(0) + 1)
print(f'Cо spacy tokenizer:')
print(f'Всего {sum_zero} нулевых векторов в вопросах')
for k in [1, 5, 10, 100, 500, 1000]:
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(sp_ranking, k), k, hits_count(sp_ranking, k)))

print()

  0%|          | 0/3760 [00:00<?, ?it/s]

Cо spacy tokenizer:
Всего 1 нулевых векторов в вопросах
DCG@   1: 0.513 | Hits@   1: 0.513
DCG@   5: 0.597 | Hits@   5: 0.673
DCG@  10: 0.618 | Hits@  10: 0.737
DCG@ 100: 0.651 | Hits@ 100: 0.893
DCG@ 500: 0.661 | Hits@ 500: 0.976
DCG@1000: 0.664 | Hits@1000: 1.000



В целом отметим, что лучший результат показал WP-токенизатор. Хотя к нему достаточно близко оказались дефолтный и spacy-токенизаторы. А вот вторая версия токенизатора с постпроцессингом показала себя намного хуже всех остальных. Похоже, что процедуры постпроцессинга (удаление стоп-слов, знаков пунктуации, стемминг) не только не возымели эффекта, но и негативно сказались на качестве ранжирования. 

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


В проведённых экспериментах наилучшее качество показал `WordPunkt` токенизатор с токенизацией знаков пунктуации, чуть хуже себя показали дефолтный токенизатор на основе регулярных выражений и `Spacy`, на последнем месте вторая версия токенизатора с постпроцессингом. Такие результаты можно объяснить тем что, для вопросов и ответов на StackOverFlow не так важны знаки препинания (это не художественный текст) и формы слов, сколько семантика. Данный тип текстов скорее технические диалоги в виде свободной формы общения, которые могут позволять себе некоторые вольности. Конечно, на другом корпусе вопросов (например на корпусе текстов художественной литературы) результат может быть полностью противоположный.

С учётом вышесказанного, эмбеддинги без нормализации показали себя лучше чем эмбеддинги созданные с предварительной нормализацией (лемматизацией) слов. Если сравнить предобученные эмбеддинги с обученными, то последние показали точность выше как на `cbow` так и, особенно, на `skip-gram` не смотря почти на вдвое меньший объём векторного пространства. Я связываю это с подбором оптимальных параметров `window` и `min_count` для текущнй задачи, что позволило увеличить итоговую точность. Однако скорее всего, стоило брать `min_count < 6`, например `min_count = 5`, т.к. при данном значении тоже был продемонстрирован хороший результат, не сильно отстающий от лучшего. 

Итоговый результат получился не очень впечатляющий (максимальная точность `DCG@1: 0.517`, `Hits@1: 0.517`). Вероятно это связано с тем, что `Word2Vec` подход достаточно устаревший и имеет ряд существенных недостатков, таких как - отсутствие информации о предложении или контексте, в котором используется слово; не учитывается совместная встречаемость слов и разное значение одного и того же слова в зависимости от контекста. В качестве улучшения можно использовать, например, модель `GloVe` (Global Vectors), которая учитывает частоту встречаемость слов и опережает Word2Vec в большинстве тестов. Или библиотеку `fastText`, которая к основной модели Word2Vec добавляет модель символьных n-грамм и способна генерировать эмбеддинги и для неизвестных слов. Это не говоря уже о таких методах как реккурентные нейронные сети, трансформеры, языковые модели GPT, BERT.